In [3]:
# Cellule d'installation des bibliothèques
!pip install tensorflow opencv-python-headless matplotlib numpy scikit-learn -q
!pip install mlflow -q

In [5]:
# Accéder au secret
from google.colab import userdata
import os

GITHUB_TOKEN = userdata.get('GITHUB_TOKEN')
GITHUB_USER = 'M-Carre'
GITHUB_REPO_NAME = 'OC-autonomous-vehicle-segmentation'
CLONE_DIR = f"/content/{GITHUB_REPO_NAME}"

# Construire l'URL de clonage
clone_url = f"https://{GITHUB_TOKEN}@github.com/{GITHUB_USER}/{GITHUB_REPO_NAME}.git"

# Exécuter la commande git clone
# os.system() est parfois plus propre pour ça que ! avec f-strings complexes
os.system(f"git clone {clone_url} {CLONE_DIR}")

# Vérifier
if os.path.exists(CLONE_DIR):
    %cd {CLONE_DIR}
    !ls
    print(f"Dépôt cloné avec succès dans {CLONE_DIR}")
else:
    print(f"Échec du clonage du dépôt dans {CLONE_DIR}")

/content/OC-autonomous-vehicle-segmentation
README.md
Dépôt cloné avec succès dans /content/OC-autonomous-vehicle-segmentation


# Script GitHub

In [13]:
# ==============================================================================
# SCRIPT DE SYNCHRONISATION (DRIVE -> REPO LOCAL) ET PUSH VERS GITHUB
# ==============================================================================
import os
import shutil
from google.colab import userdata # Pour le token (au cas où, mais normalement pas directement pour git push si remote est bien configuré)

# --- Configuration des Chemins et Fichiers ---

# 1. Chemin vers le dossier racine de votre projet sur Google Drive
#    (celui où vous éditez vos fichiers de code/notebooks)
PROJECT_BASE_ON_DRIVE = '/content/drive/MyDrive/Colab Notebooks/OC/P8'

# 2. Chemin vers votre dépôt Git cloné dans l'environnement d'exécution de Colab
GIT_REPO_PATH_IN_COLAB = '/content/OC-autonomous-vehicle-segmentation'

# 3. Nom de la branche Git pour le push
GIT_BRANCH_NAME = 'main' # Ou 'master', ou la branche sur laquelle vous travaillez

# 4. LISTE DES FICHIERS ET DOSSIERS À SYNCHRONISER DEPUIS DRIVE VERS LE REPO LOCAL
#    Spécifiez les chemins RELATIFS à PROJECT_BASE_ON_DRIVE.
#    IMPORTANT : NE PAS LISTER LES DOSSIERS CONTENANT LES GROSSES DONNÉES CITYSCAPES !
ITEMS_TO_SYNC = [
    "notebooks/Notebook_modélisation.ipynb", # Fichier individuel
    "src",                                   # Dossier entier (sera copié récursivement)
    "scripts",                               # Dossier entier
    ".gitignore",                            # Fichier individuel (si vous l'éditez sur Drive)
    "README.md",                             # Fichier individuel (si vous l'éditez sur Drive)
    "api",
    "webapp",
    "reports",
    "models", # Attention si vous mettez des fichiers de modèles ici, préférez MLflow pour les gros modèles
]
# ------------------------------------------------------------------------------

def sync_and_push_to_github():
    print("### 1. VÉRIFICATION DES CHEMINS ###")
    if not os.path.exists(PROJECT_BASE_ON_DRIVE):
        print(f"ERREUR : Le dossier source sur Drive '{PROJECT_BASE_ON_DRIVE}' n'existe pas.")
        return
    if not os.path.exists(GIT_REPO_PATH_IN_COLAB):
        print(f"ERREUR : Le dossier du dépôt Git cloné '{GIT_REPO_PATH_IN_COLAB}' n'existe pas.")
        return
    print(f"Source du projet sur Drive : {PROJECT_BASE_ON_DRIVE}")
    print(f"Dépôt Git local dans Colab : {GIT_REPO_PATH_IN_COLAB}")

    print("\n### 2. SYNCHRONISATION DES FICHIERS (DRIVE -> REPO LOCAL) ###")
    synced_item_count = 0
    for item_rel_path in ITEMS_TO_SYNC:
        source_item_abs_path = os.path.join(PROJECT_BASE_ON_DRIVE, item_rel_path)
        dest_item_abs_path = os.path.join(GIT_REPO_PATH_IN_COLAB, item_rel_path)

        if not os.path.exists(source_item_abs_path):
            print(f"AVERTISSEMENT : L'élément source '{source_item_abs_path}' n'existe pas sur Drive. Ignoré.")
            continue

        # S'assurer que le dossier parent de destination existe (pour les fichiers dans des sous-dossiers)
        os.makedirs(os.path.dirname(dest_item_abs_path), exist_ok=True)

        if os.path.isdir(source_item_abs_path):
            # Pour les dossiers, supprimer la destination si elle existe et la recopier
            if os.path.exists(dest_item_abs_path):
                print(f"   Suppression du dossier de destination existant : {dest_item_abs_path}")
                shutil.rmtree(dest_item_abs_path)
            print(f"   Copie du dossier : '{source_item_abs_path}' vers '{dest_item_abs_path}'")
            shutil.copytree(source_item_abs_path, dest_item_abs_path)
            synced_item_count += 1
        elif os.path.isfile(source_item_abs_path):
            print(f"   Copie du fichier : '{source_item_abs_path}' vers '{dest_item_abs_path}'")
            shutil.copy2(source_item_abs_path, dest_item_abs_path)
            synced_item_count += 1
        else:
            print(f"AVERTISSEMENT : L'élément source '{source_item_abs_path}' n'est ni un fichier ni un dossier. Ignoré.")

    if synced_item_count == 0 and ITEMS_TO_SYNC:
        print("Aucun des éléments listés dans ITEMS_TO_SYNC n'a été trouvé sur Drive pour la synchronisation.")
    elif synced_item_count > 0 :
        print(f"{synced_item_count} élément(s) synchronisé(s) avec succès.")

    print("\n### 3. OPÉRATIONS GIT ###")
    try:
        os.chdir(GIT_REPO_PATH_IN_COLAB)
        print(f"Changé de répertoire vers : {os.getcwd()}")
    except FileNotFoundError:
        print(f"ERREUR CRITIQUE : Impossible de naviguer vers le répertoire du dépôt Git '{GIT_REPO_PATH_IN_COLAB}'.")
        return

    # --- Configuration de l'identité Git (si pas déjà fait dans la session) ---
    # Décommentez et remplissez si nécessaire
    # print("Configuration de l'identité Git...")
    # !git config --global user.email "VOTRE_EMAIL_GITHUB@example.com"
    # !git config --global user.name "VOTRE_NOM_UTILISATEUR_GITHUB"

    print("\n--- Statut Git (après synchronisation) ---")
    !git status

    print("\nAjout de tous les changements au staging area (git add .)...")
    !git add . # Cela ajoutera tous les changements dans le dépôt cloné,
               # y compris les fichiers copiés et les suppressions si des fichiers
               # ont été retirés de ITEMS_TO_SYNC et supprimés manuellement du repo.

    commit_message = input("\nEntrez votre message de commit (laissez vide pour annuler le commit/push) : ")

    if not commit_message:
        print("\nMessage de commit vide. Opération de commit et push annulée.")
        # Afficher à nouveau le statut pour que l'utilisateur voie les changements en attente
        print("\n--- Statut Git Actuel (changements non commités) ---")
        !git status
        return

    print(f"\nCommit des changements avec le message : '{commit_message}'")
    os.environ['GIT_COMMIT_MESSAGE'] = commit_message
    !git commit -m "$GIT_COMMIT_MESSAGE"
    del os.environ['GIT_COMMIT_MESSAGE']

    print(f"\nPush des changements vers la branche '{GIT_BRANCH_NAME}' sur GitHub...")
    !git push origin {GIT_BRANCH_NAME}

    print("\n--- Opération de synchronisation, commit et push terminée. ---")
    print("Vérifiez votre dépôt sur GitHub pour confirmer les changements.")

# --- Exécution de la fonction ---
sync_and_push_to_github()

Copié : '/content/drive/MyDrive/Colab Notebooks/OC/P8/notebooks/Notebook_modélisation.ipynb' vers '/content/OC-autonomous-vehicle-segmentation/notebooks/Notebook_modélisation.ipynb'
-rw------- 1 root root 4.5K May 15 13:40 /content/OC-autonomous-vehicle-segmentation/notebooks/Notebook_modélisation.ipynb
